In [1]:
import pandas as pd
import geopandas as gpd

In [2]:
adressen_df = pd.read_csv("raw_data/adressen_aktuell.txt")

In [3]:
adressen_df.head()

,STRASSE,STRANAM,HSZ
0,900,Donaustr.,32
1,1733,Häusserstr.,21
2,2010,Hitzestr.,7
3,2010,Hitzestr.,11
4,2053,Höslstr.,5


In [4]:
adressen_count_df = adressen_df.groupby(["STRASSE"]).agg({"STRANAM": "first", "HSZ":"size"}).sort_values("HSZ",ascending=False)

In [5]:
adressen_count_df.head(10)

,STRANAM,HSZ
STRASSE,,
4166,Schleißheimer Str.,499
2690,Landsberger Str.,426
1129,Eversbuschstr.,364
1249,Forstenrieder Allee,359
797,Dachauer Str.,349
2781,Lerchenauer Str.,336
400,Balanstr.,335
4795,Truderinger Str.,324
1179,Feldmochinger Str.,321


In [6]:
strassen_gdf = gpd.read_file("raw_data/ows.gml").set_crs("EPSG:25832", allow_override=True)

In [7]:
dissolved_adressen_gdf = strassen_gdf.dissolve(by="str_nr")["geometry"]

In [8]:
dissolved_adressen_gdf

str_nr
0.0        MULTILINESTRING ((696652.759 5338508.83, 69665...
1.0        LINESTRING (691774.732 5338190.733, 691767.155...
2.0        MULTILINESTRING ((697763.628 5338374.322, 6977...
3.0        LINESTRING (694347.125 5337278.02, 694356.554 ...
4.0        MULTILINESTRING ((687533.315 5339244.495, 6875...
                                 ...                        
30033.0    LINESTRING (683389.744 5340635.2, 683470.635 5...
30035.0    LINESTRING (688742.093 5338715.906, 688735.475...
30043.0    MULTILINESTRING ((688684.503 5331097.516, 6887...
40008.0    LINESTRING (694628.642 5331151.454, 694621.737...
90004.0    LINESTRING (678085.163 5338372.419, 678078.847...
Name: geometry, Length: 6143, dtype: geometry

In [9]:
combined_df = gpd.GeoDataFrame(adressen_count_df.join(dissolved_adressen_gdf, on="STRASSE"))

In [10]:
combined_df["laenge"] = combined_df.geometry.length

In [11]:
combined_df.sort_values("laenge", ascending=False).head(10)

,STRANAM,HSZ,geometry,laenge
STRASSE,,,,
797,Dachauer Str.,349,"MULTILINESTRING ((690242.92 5336239.553, 69024...",19705.708937
4166,Schleißheimer Str.,499,"MULTILINESTRING ((690417.552 5342683.97, 69041...",14158.416756
2690,Landsberger Str.,426,"MULTILINESTRING ((687365.963 5335080.632, 6872...",13752.935779
2694,Landshuter Allee,127,"MULTILINESTRING ((688791.082 5339324.934, 6887...",13068.731376
1376,Fürstenrieder Str.,280,"MULTILINESTRING ((686211.123 5335120.858, 6862...",11239.526237
4673,Tegernseer Landstr.,246,"MULTILINESTRING ((693214.99 5329099.366, 69319...",10890.674817
4505,Ständlerstr.,50,"MULTILINESTRING ((696293.522 5331529.352, 6963...",9295.938996
5669,Georg-Brauchle-Ring,26,"MULTILINESTRING ((688729.513 5339080.06, 68873...",8109.038751
294,Arnulfstr.,185,"MULTILINESTRING ((688125.269 5335768.855, 6881...",8047.039901


In [12]:
combined_df.sort_values("HSZ", ascending=False).head(10)

,STRANAM,HSZ,geometry,laenge
STRASSE,,,,
4166,Schleißheimer Str.,499,"MULTILINESTRING ((690417.552 5342683.97, 69041...",14158.416756
2690,Landsberger Str.,426,"MULTILINESTRING ((687365.963 5335080.632, 6872...",13752.935779
1129,Eversbuschstr.,364,"MULTILINESTRING ((682656.262 5340961.143, 6826...",4010.817865
1249,Forstenrieder Allee,359,"MULTILINESTRING ((685937.268 5329422.612, 6859...",3294.989680
797,Dachauer Str.,349,"MULTILINESTRING ((690242.92 5336239.553, 69024...",19705.708937
2781,Lerchenauer Str.,336,"MULTILINESTRING ((689651.29 5340124.422, 68963...",7741.370127
400,Balanstr.,335,"MULTILINESTRING ((694468.173 5329438.678, 6944...",4805.549772
4795,Truderinger Str.,324,"MULTILINESTRING ((696783.426 5334246.731, 6967...",5116.521044
1179,Feldmochinger Str.,321,"MULTILINESTRING ((687831.235 5343287.463, 6878...",5008.575938


In [13]:
import plotly.express as px
px.scatter(combined_df, x="laenge", y="HSZ", hover_data=["STRANAM"])

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'customdata': array([['Schleißheimer Str.'],
                                   ['Landsberger Str.'],
                                   ['Eversbuschstr.'],
                                   ...,
                                   ['Hans-Bieringer-Weg'],
                                   ['Wiguläus-Hundt-Weg'],
                                   ['Fritz-Neuland-Str.']], shape=(6301, 1), dtype=object),
              'hovertemplate': 'laenge=%{x}<br>HSZ=%{y}<br>STRANAM=%{customdata[0]}<extra></extra>',
              'legendgroup': '',
              'marker': {'color': '#636efa', 'symbol': 'circle'},
              'mode': 'markers',
              'name': '',
              'showlegend': False,
              'type': 'scattergl',
              'x': {'bdata': ('gChEWDWny0DnipzHd9zKQDhoI7+iVa' ... 'BqWbpWQIOYI99EHFZAav0MhxnqREA='),
                    'dtype': 'f8'},
              'xaxis': 'x',
              'y': {'bdata': ('8wGqAWwBZwFdAVABTwFEAUEBLwEYAR' ... 'EAAQABAAEAAQABAAEAAQABAAEAAQA='),
                    'dtype': 'i2'},
              'yaxis': 'y'}],
    'layout': {'legend': {'tracegroupgap': 0},
               'margin': {'t': 60},
               'template': '...',
               'xaxis': {'anchor': 'y', 'domain': [0.0, 1.0], 'title': {'text': 'laenge'}},
               'yaxis': {'anchor': 'x', 'domain': [0.0, 1.0], 'title': {'text': 'HSZ'}}}
})

In [14]:
combined_df["adressen_pro_100meter"] = (combined_df["HSZ"] / combined_df["laenge"]) * 100

In [15]:
combined_df.sort_values("adressen_pro_100meter", ascending=False).head(10)

,STRANAM,HSZ,geometry,laenge,adressen_pro_100meter
STRASSE,,,,,
6021,Seiffertstr.,98,"LINESTRING (697855.331 5336044.186, 697856.359...",102.855332,95.279455
1950,Hertwigpl.,11,"LINESTRING (683800.173 5340162.234, 683812.278...",14.515148,75.782899
1470,Georg-Wopfner-Str.,40,"MULTILINESTRING ((694135.514 5340649.076, 6941...",57.324496,69.778197
6346,Karl-Breu-Weg,51,"MULTILINESTRING ((697591.62 5334443.574, 69753...",89.624841,56.903866
6050,Von-der-Vring-Str.,32,"LINESTRING (696950.368 5335529.519, 696911.475...",57.134013,56.008669
379,Bad-Kissingen-Str.,94,"MULTILINESTRING ((695725.1 5333039.303, 695722...",177.941721,52.826285
6003,Edmund-Husserl-Str.,24,"LINESTRING (681150.049 5337463.595, 681145.779...",49.088773,48.891016
6007,Martin-Heidegger-Str.,32,"LINESTRING (681120.931 5337412.566, 681147.555...",74.184674,43.135594
2732,Lechbrucker Str.,43,"MULTILINESTRING ((685470.236 5329080.362, 6854...",112.861804,38.099692


In [16]:
combined_df[combined_df["laenge"]>1000].sort_values("adressen_pro_100meter", ascending=False).head(10)

,STRANAM,HSZ,geometry,laenge,adressen_pro_100meter
STRASSE,,,,,
4244,Schrämelstr.,182,"MULTILINESTRING ((684010.811 5336973.765, 6839...",1127.702701,16.139005
1869,Heilwigstr.,165,"MULTILINESTRING ((699657.546 5332457.12, 69964...",1110.732213,14.855066
4212,Schneeglöckchenstr.,182,"MULTILINESTRING ((686514.612 5341521.656, 6863...",1281.530789,14.201766
21,Adam-Berg-Str.,140,"MULTILINESTRING ((695356.234 5331469.776, 6953...",1041.171727,13.446389
1123,Eulenspiegelstr.,147,"MULTILINESTRING ((698475.688 5328545.964, 6984...",1125.157163,13.064842
2056,Hofangerstr.,231,"MULTILINESTRING ((695716.662 5332474.524, 6957...",1818.804397,12.700651
5096,Wertheimer Str.,143,"MULTILINESTRING ((680258.971 5336012.921, 6802...",1128.339223,12.673494
4800,Tsingtauer Str.,164,"MULTILINESTRING ((699957.332 5330984.915, 6999...",1304.052773,12.576178
3434,Ottilienstr.,140,"MULTILINESTRING ((699456.753 5332265.995, 6994...",1118.141294,12.520779


In [17]:
combined_df.sort_values("adressen_pro_100meter", ascending=True).head(20)

,STRANAM,HSZ,geometry,laenge,adressen_pro_100meter
STRASSE,,,,,
6251,Max-Born-Str.,1,"MULTILINESTRING ((688062.513 5340226.578, 6880...",4588.017325,0.021796
5811,Donnersbergerbrücke,1,"MULTILINESTRING ((688578.056 5335621.092, 6885...",2746.062841,0.036416
6744,Heckenstallertunnel,1,"MULTILINESTRING ((688626.022 5331682.119, 6886...",2564.726514,0.038991
6503,Hans-Steinkohl-Str.,1,"MULTILINESTRING ((679290.222 5334304.147, 6792...",2216.864236,0.045109
2159,Ifflandstr.,2,"MULTILINESTRING ((693572.308 5337098.187, 6935...",4395.818121,0.045498
6461,Isartalbahnweg,1,"MULTILINESTRING ((688929.016 5328563.95, 68891...",2039.671176,0.049028
3501,Pellegrinistr.,1,"MULTILINESTRING ((698019.037 5336700.277, 6980...",1822.632922,0.054866
6661,Weißenhofweg,1,"MULTILINESTRING ((692473.755 5339443.664, 6924...",1731.102029,0.057767
5522,Lippweg,1,"MULTILINESTRING ((681830.351 5341956.979, 6818...",1705.413077,0.058637


In [36]:
combined_df[combined_df["HSZ"]==1].sort_values("laenge", ascending=False).head(10)

,STRANAM,HSZ,geometry,laenge,adressen_pro_100meter
STRASSE,,,,,
6251,Max-Born-Str.,1,"MULTILINESTRING ((688062.513 5340226.578, 6880...",4588.017325,0.021796
5811,Donnersbergerbrücke,1,"MULTILINESTRING ((688578.056 5335621.092, 6885...",2746.062841,0.036416
6744,Heckenstallertunnel,1,"MULTILINESTRING ((688626.022 5331682.119, 6886...",2564.726514,0.038991
6503,Hans-Steinkohl-Str.,1,"MULTILINESTRING ((679290.222 5334304.147, 6792...",2216.864236,0.045109
6461,Isartalbahnweg,1,"MULTILINESTRING ((688929.016 5328563.95, 68891...",2039.671176,0.049028
3501,Pellegrinistr.,1,"MULTILINESTRING ((698019.037 5336700.277, 6980...",1822.632922,0.054866
6661,Weißenhofweg,1,"MULTILINESTRING ((692473.755 5339443.664, 6924...",1731.102029,0.057767
5522,Lippweg,1,"MULTILINESTRING ((681830.351 5341956.979, 6818...",1705.413077,0.058637
5581,An der Schloßmauer,1,"MULTILINESTRING ((685539.398 5335811.373, 6855...",1656.842777,0.060356


In [20]:
combined_df[combined_df["STRANAM"]=="Dorfstr."]

,STRANAM,HSZ,geometry,laenge,adressen_pro_100meter
STRASSE,,,,,
904,Dorfstr.,43,"MULTILINESTRING ((683442.918 5337557.837, 6834...",918.415761,4.681975


In [41]:
combined_df[combined_df["STRANAM"]=="Platz der Opfer des Nationalsozialismus"]

,STRANAM,HSZ,geometry,laenge,adressen_pro_100meter
STRASSE,,,,,
3617,Platz der Opfer des Nationalsozialismus,1,"LINESTRING (691424.709 5335350.029, 691438.69 ...",98.384249,1.016423


In [25]:
bezrike_gdf =gpd.read_file("processed_data/muenchen_stadtbezirke.geojson")
sjoined_gdf = combined_df.sjoin(bezrike_gdf, how="left", predicate="intersects")
sjoined_gdf.head()

,STRANAM,HSZ,geometry,laenge,adressen_pro_100meter,index_right,sb_nummer,sb_name
STRASSE,,,,,,,,
4166,Schleißheimer Str.,499,"MULTILINESTRING ((690417.552 5342683.97, 69041...",14158.416756,3.524405,2.0,03,Maxvorstadt
4166,Schleißheimer Str.,499,"MULTILINESTRING ((690417.552 5342683.97, 69041...",14158.416756,3.524405,3.0,04,Schwabing-West
4166,Schleißheimer Str.,499,"MULTILINESTRING ((690417.552 5342683.97, 69041...",14158.416756,3.524405,10.0,11,Milbertshofen-Am Hart
4166,Schleißheimer Str.,499,"MULTILINESTRING ((690417.552 5342683.97, 69041...",14158.416756,3.524405,23.0,24,Feldmoching-Hasenbergl
2690,Landsberger Str.,426,"MULTILINESTRING ((687365.963 5335080.632, 6872...",13752.935779,3.097520,1.0,02,Ludwigsvorstadt-Isarvorstadt


In [26]:
sjoined_gdf.groupby("sb_name").size().sort_values(ascending=False)

sb_name
Bogenhausen                                               466
Thalkirchen-Obersendling-Forstenried-Fürstenried-Solln    448
Ramersdorf-Perlach                                        446
Pasing-Obermenzing                                        441
Trudering-Riem                                            407
Schwabing-Freimann                                        392
Aubing-Lochhausen-Langwied                                345
Neuhausen-Nymphenburg                                     320
Untergiesing-Harlaching                                   288
Feldmoching-Hasenbergl                                    271
Allach-Untermenzing                                       270
Sendling-Westpark                                         243
Milbertshofen-Am Hart                                     228
Moosach                                                   216
Hadern                                                    212
Altstadt-Lehel                                            209


In [40]:
sjoined_gdf.groupby(level=0).agg({"sb_name": "size", "STRANAM": "first"}).sort_values("sb_name", ascending=False)

,sb_name,STRANAM
STRASSE,,
797,5,Dachauer Str.
1376,4,Fürstenrieder Str.
2690,4,Landsberger Str.
4166,4,Schleißheimer Str.
1455,3,Georgenstr.
...,...,...
2359,1,Karl-Schurz-Str.
2358,1,Karl-Schmolz-Str.
2357,1,Karlsburger Str.


In [31]:
sjoined_gdf.groupby(level=0).size().sort_values(ascending=False).plot(kind="hist", bins=5, backend='plotly')

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'bingroup': 'x',
              'hovertemplate': 'variable=0<br>value=%{x}<br>count=%{y}<extra></extra>',
              'legendgroup': '0',
              'marker': {'color': '#636efa', 'pattern': {'shape': ''}},
              'name': '0',
              'orientation': 'v',
              'showlegend': True,
              'type': 'histogram',
              'x': {'bdata': ('BQQEBAMDAwMDAwMDAwMDAwMDAwMDAw' ... 'EBAQEBAQEBAQEBAQEBAQEBAQEBAQ=='),
                    'dtype': 'i1'},
              'xaxis': 'x',
              'yaxis': 'y'}],
    'layout': {'barmode': 'relative',
               'legend': {'title': {'text': 'variable'}, 'tracegroupgap': 0},
               'margin': {'t': 60},
               'template': '...',
               'xaxis': {'anchor': 'y', 'domain': [0.0, 1.0], 'title': {'text': 'value'}},
               'yaxis': {'anchor': 'x', 'domain': [0.0, 1.0], 'title': {'text': 'count'}}}
})

In [34]:
combined_df.groupby("STRANAM").size().sort_values(ascending=True).head(10)

STRANAM
ALEXANDER-MIKLÓSY-W.    1
Pasinger Heuweg         1
Pasinger Bahnhofspl.    1
Pasettiweg              1
Paschstr.               1
Parzivalstr.            1
Parzivalpl.             1
Partnachstr.            1
Partnachpl.             1
Partenkirchner Str.     1
dtype: int64

In [37]:
combined_df[combined_df["STRANAM"].str.contains("roß")]

,STRANAM,HSZ,geometry,laenge,adressen_pro_100meter
STRASSE,,,,,
1649,Großfriedrichsburger Str.,94,"MULTILINESTRING ((701521.819 5332681.119, 7015...",631.043576,14.895960
1652,Großhaderner Str.,84,"MULTILINESTRING ((684514.355 5332098.498, 6845...",911.347119,9.217125
1656,Großvenedigerstr.,46,"MULTILINESTRING ((696425.481 5332997.444, 6964...",410.624135,11.202459
1653,Großhesseloher Str.,27,"MULTILINESTRING ((688563.006 5328547.026, 6885...",470.748519,5.735546
1655,Groß-Nabas-Str.,18,"LINESTRING (701477.962 5331961.521, 701483.262...",209.834942,8.578171
1648,Großbeerenstr.,15,"LINESTRING (686346.2 5339265.284, 686451.264 5...",124.810788,12.018192
1654,Großlappen,1,None,NaN,NaN


In [18]:
combined_df.to_file("processed_data/strassen_adressen_muenchen.geojson")

In [19]:
gdf_wgs84 = combined_df.to_crs(epsg=4326)
gdf_wgs84.to_file("processed_data/strassen_adressen_muenchen_wgs84.geojson", driver="GeoJSON")